## NBA Web Scraping - Parsing Box Scores Into Pandas DataFrame

### i.) Importing Libraries, Dependenies & Defining box_score File Path to Parse HTML

In [1]:
#Import libraries and dependencies 
import os
import pandas as pd
from bs4 import BeautifulSoup
import html5lib

In [2]:
#Define score directory variable where scores are stored
#Note: Box scores dir may be empty due to nba_predictions_2.py timeout errors when attempting to scrap data
#If directory empty, create Dataframe directly from nba_games.csv

SCORE_DIR = "data2/scores"

In [3]:
box_scores = os.listdir(SCORE_DIR)

In [4]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [5]:
box_scores

['data2/scores/202110200POR.html',
 'data2/scores/202110240SAC.html',
 'data2/scores/202110220PHI.html',
 'data2/scores/202110240LAL.html',
 'data2/scores/202110220ORL.html',
 'data2/scores/202110240NYK.html',
 'data2/scores/202110200CHO.html',
 'data2/scores/202110220LAL.html',
 'data2/scores/202110200MIN.html',
 'data2/scores/202110220SAC.html',
 'data2/scores/202110200TOR.html',
 'data2/scores/202110200SAS.html',
 'data2/scores/202110210GSW.html',
 'data2/scores/202110210MIA.html',
 'data2/scores/202110200DET.html',
 'data2/scores/202110210ATL.html',
 'data2/scores/202110230IND.html',
 'data2/scores/202110190LAL.html',
 'data2/scores/202110200NOP.html',
 'data2/scores/202110230LAC.html',
 'data2/scores/202110220WAS.html',
 'data2/scores/202110240HOU.html',
 'data2/scores/202110230POR.html',
 'data2/scores/202110240OKC.html',
 'data2/scores/202110240BRK.html',
 'data2/scores/202110230SAS.html',
 'data2/scores/202110190MIL.html',
 'data2/scores/202110220HOU.html',
 'data2/scores/20211

In [6]:
len(box_scores)

38

In [7]:
#i.e. viewing a single box_score html file path 
box_scores[0]

'data2/scores/202110200POR.html'

In [8]:
# Parse html function for single box_score file
# take in the path to a box score html file
# open box score file as f, and then read it into the 'html' variable instance
# then create BeautifulSoup instance to parse the data (parse = change the format of the data to another data form type)
# cleans up data for further processing

def parse_html(box_score):
    with open (box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    #soup.select selects the pieces of the parsed html that we want to remove
    #tr.over_header -> over_header is the mini header overtop of the 'Basic Box Score Stats'
    #write a list comprehension that calls s.decompose -> selects all tr.over_header that removes them from html
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [ ]:
box_score = box_scores[0]
soup = parse_html(box_score)
#i.e. print example of the soup html below if required to test
#print(soup)

### ii.) Reading Line Score With Pandas DataFrame

In [17]:
# Create a function that reads the 'Line Score' in each html file
# *Note: There is a possible problem with each 'Line Score' in the downloaded html files in 'data2/scores' for some reason
# **** Above resolved! Note: Playwright as p -> p.chromium.launch() will NOT return 'Line Score' tables as is
# **** MUST use p.firefox.launch() as a substitute -> also has less timeout issues when running for loop scrape_game

# Read 'Line Score' table of html data file
# soup is html text format returned & cleaned from parse_html function above
# pandas 'read_html' function reads in converted html soup data 
# call 'str' on str(soup) to get html text data out of soup class
# invoke 'attrs' (attributes = process table that has 'id' 'line_score') 
# look for attribute id that contains line_score, [0] first dataframe out of it as pandas returns multiple dataframes from this
# use 'cols' to repair column names of the line_score datframe

def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns=cols

    line_score = line_score[["team", "total"]]
    
    return line_score

In [9]:
box_score = box_scores[0]
box_score

'data2/scores/202110200POR.html'

In [81]:
# ------ Testing Box --------
# ***************************

# Testing producing and printing 'Line Score' for box_score[i] game 
# works (11/22/20222)
#soup = parse_html(box_score)
#line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
#cols = list(line_score.columns)
#cols[0] = "team"
#cols[-1] = "total"
#line_score.columns = cols
#print(line_score)
#print()
#print()


#box_score = box_scores[0]
#soup = parse_html(box_score)
#line_score = read_line_score(soup)
#teams = list(line_score["team"])

#summaries = []
#for team in teams:
#    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-basic"}, index_col=0)[0]

#display(df)

In [40]:
# Parse the 'Line Score' stats table and the main advanced stat table for each entire game

def read_stats (soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [60]:
line_score

,team,total
0,SAC,124
1,POR,121


In [74]:
# Read season info & season number from html (soup) data file

def read_season_info(soup): 
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")] 
    # take second href [1] in hrefs gathered list, and the first string element in the 'split' list
    season  = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [79]:
box_score = box_scores[0]
games = []
base_cols = None

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    for team in teams:
        basic  = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        # Creates single pandas 'Series' with basic & advanced statistics (1D array)
        totals = pd.concat([basic.iloc[-1, :], advanced.iloc[-1, :]])
        totals.index = totals.index.str.lower()

        #maximum value for each player
        #basic take all rows [:=1] except the last one; all columns
        maxes = pd.concat([basic.iloc[:-1, :].max(), advanced.iloc[:-1, :].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        # Create summary as combination of totals & maxes
        summary = pd.concat([totals, maxes])

        if base_cols is None: 
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols] 

        summaries.append(summary)
    # transpose (swap row <-> column) dataframe so game stats become columns
    summary = pd.concat(summaries, axis=1).T

    # Create 'game' dataframe that includes both summary stats & line_score (team names & final score) stats
    game = pd.concat([summary, line_score], axis=1)

    # Assign 'away' [0] and 'home' [1] teams in final transposed game database (first indexed row team [0] is always away, [1] second row is home [1])
    game["home"] = [0, 1] 

    # Next, build a mirrored-esque dataframe for the opponents 'game' data
    # Reverse the index-row order of the game dataframe (i.e. data.iloc[::-1])
    game_opp = game.iloc[::-1].reset_index()
    # Rename all the original 'game' columns with a '_opp' suffix naming convention
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["season"] = read_season_info(soup)

    # takes the date as the first 8 char of the box_score file name and inserts into newly created 'date' col
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    full_game["won"] = full_game["total"]> full_game["total_opp"]

    games.append(full_game)

In [42]:
advanced

,MP,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
Starters,,,,,,,,,,,,,,,,
Damian Lillard,NaN,0.382,0.333,0.375,0.208,0.0,15.2,7.8,37.0,1.1,0.0,3.7,28.3,99.0,120.0,-6.2
CJ McCollum,NaN,0.708,0.708,0.458,0.000,6.1,11.6,8.9,26.3,3.9,2.7,17.2,34.5,118.0,112.0,10.9
Norman Powell,NaN,0.559,0.389,0.333,0.889,0.0,14.5,7.4,0.0,0.0,0.0,7.4,16.1,110.0,123.0,-8.7
Robert Covington,NaN,0.444,0.444,0.778,0.000,0.0,13.2,6.8,0.0,0.0,6.1,10.0,13.5,79.0,118.0,-6.8
Jusuf Nurkić,NaN,0.765,0.650,0.100,0.700,11.1,39.1,25.4,9.5,0.0,3.3,7.1,20.5,155.0,116.0,6.0
Cody Zeller,NaN,0.644,0.667,0.000,0.667,11.1,21.4,16.4,6.3,2.4,4.9,0.0,8.5,153.0,112.0,1.9
Anfernee Simons,NaN,0.917,0.917,0.333,0.000,0.0,11.6,5.9,34.6,0.0,0.0,14.3,16.6,160.0,123.0,7.5
Larry Nance Jr.,NaN,0.333,0.333,0.333,0.000,0.0,17.9,9.1,13.3,0.0,0.0,25.0,9.8,73.0,122.0,-11.3
Nassir Little,NaN,0.700,0.700,0.200,0.000,13.5,6.5,9.9,0.0,0.0,0.0,16.7,15.9,122.0,124.0,-4.0


In [44]:
basic

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
Starters,,,,,,,,,,,,,,,,,,,,
Damian Lillard,NaN,8.0,24.0,0.333,0.0,9.0,0.000,4.0,5.0,0.800,0.0,6.0,6.0,11.0,1.0,0.0,1.0,3.0,20.0,-2.0
CJ McCollum,NaN,14.0,24.0,0.583,6.0,11.0,0.545,0.0,0.0,NaN,2.0,4.0,6.0,5.0,3.0,1.0,5.0,4.0,34.0,0.0
Norman Powell,NaN,3.0,9.0,0.333,1.0,3.0,0.333,7.0,8.0,0.875,0.0,5.0,5.0,0.0,0.0,0.0,1.0,3.0,14.0,3.0
Robert Covington,NaN,3.0,9.0,0.333,2.0,7.0,0.286,0.0,0.0,NaN,0.0,4.0,4.0,0.0,0.0,2.0,1.0,2.0,8.0,-9.0
Jusuf Nurkić,NaN,6.0,10.0,0.600,1.0,1.0,1.000,7.0,7.0,1.000,3.0,11.0,14.0,2.0,0.0,1.0,1.0,4.0,20.0,-2.0
Cody Zeller,NaN,2.0,3.0,0.667,0.0,0.0,NaN,1.0,2.0,0.500,2.0,4.0,6.0,1.0,1.0,1.0,0.0,1.0,5.0,1.0
Anfernee Simons,NaN,5.0,6.0,0.833,1.0,2.0,0.500,0.0,0.0,NaN,0.0,2.0,2.0,4.0,0.0,0.0,1.0,0.0,11.0,-9.0
Larry Nance Jr.,NaN,1.0,3.0,0.333,0.0,1.0,0.000,0.0,0.0,NaN,0.0,3.0,3.0,2.0,0.0,0.0,1.0,4.0,2.0,4.0
Nassir Little,NaN,3.0,5.0,0.600,1.0,1.0,1.000,0.0,0.0,NaN,2.0,1.0,3.0,0.0,0.0,0.0,1.0,1.0,7.0,-1.0


In [48]:
maxes

mp_max          NaN
fg_max       14.000
fga_max      24.000
fg%_max       0.833
3p_max        6.000
3pa_max      11.000
3p%_max       1.000
ft_max        7.000
fta_max       8.000
ft%_max       1.000
orb_max       3.000
drb_max      11.000
trb_max      14.000
ast_max      11.000
stl_max       3.000
blk_max       2.000
tov_max       5.000
pf_max        4.000
pts_max      34.000
+/-_max       4.000
mp_max          NaN
ts%_max       0.917
efg%_max      0.917
3par_max      0.778
ftr_max       0.889
orb%_max     13.500
drb%_max     39.100
trb%_max     25.400
ast%_max     37.000
stl%_max      3.900
blk%_max      6.100
tov%_max     25.000
usg%_max     34.500
ortg_max    160.000
drtg_max    124.000
bpm_max      10.900
dtype: float64

In [52]:
totals

mp      240.000
fg       45.000
fga      93.000
fg%       0.484
3p       12.000
3pa      35.000
3p%       0.343
ft       19.000
fta      22.000
ft%       0.864
orb       9.000
drb      40.000
trb      49.000
ast      25.000
stl       5.000
blk       5.000
tov      12.000
pf       22.000
pts     121.000
+/-         NaN
mp      240.000
ts%       0.589
efg%      0.548
3par      0.376
ftr       0.237
orb%     20.000
drb%     85.100
trb%     53.300
ast%     55.600
stl%      4.800
blk%      9.800
tov%     10.500
usg%    100.000
ortg    115.700
drtg    118.600
bpm         NaN
Name: Team Totals, dtype: float64

In [54]:
summary

mp          240.000
fg           45.000
fga          93.000
fg%           0.484
3p           12.000
             ...   
tov%_max     25.000
usg%_max     34.500
ortg_max    160.000
drtg_max    124.000
bpm_max      10.900
Length: 72, dtype: float64

In [63]:
summary

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max
0,240.0,240.0,42.0,92.0,0.457,17.0,41.0,0.415,23.0,29.0,...,14.0,25.6,19.7,35.4,3.5,5.7,16.7,36.4,175.0,120.0
1,240.0,240.0,45.0,93.0,0.484,12.0,35.0,0.343,19.0,22.0,...,13.5,39.1,25.4,37.0,3.9,6.1,25.0,34.5,160.0,124.0


In [59]:
line_score

,team,total
0,SAC,124
1,POR,121


In [65]:
game

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,team,total
0,240.0,240.0,42.0,92.0,0.457,17.0,41.0,0.415,23.0,29.0,...,19.7,35.4,3.5,5.7,16.7,36.4,175.0,120.0,SAC,124
1,240.0,240.0,45.0,93.0,0.484,12.0,35.0,0.343,19.0,22.0,...,25.4,37.0,3.9,6.1,25.0,34.5,160.0,124.0,POR,121


In [80]:
full_game

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,42.0,92.0,0.457,17.0,41.0,0.415,23.0,29.0,...,25.0,34.5,160.0,124.0,POR,121,1,2022,2021-10-20,True
1,240.0,240.0,45.0,93.0,0.484,12.0,35.0,0.343,19.0,22.0,...,16.7,36.4,175.0,120.0,SAC,124,0,2022,2021-10-20,False
